# OA strikes back!

## Import

In [1]:
import os
import sys
sys.path.append('/Users/efraflores/Desktop/hub/diplo/venv/lib/python3.9/site-packages')

BASE_DIR = '/Users/efraflores/Desktop/EF/Diplo/data/05/0505'
print(sorted(os.listdir(BASE_DIR)))

['.DS_Store', 'FloresHernandezEfrainIsmael_DefaultPayment.csv', 'FloresHernandezEfrainIsmael_PAYAMT1.csv', 'ann.csv', 'logreg.csv', 'train_PAY_AMT1.csv', 'train_default.csv', 'val_PAY_AMT1.csv', 'val_default.csv', 'xgb.csv']


In [2]:
import pandas as pd

df = pd.read_csv(os.path.join(BASE_DIR,'train_PAY_AMT1.csv'),sep='|').set_index('ACCOUNT_ID')
df.sample()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
ACCOUNT_ID,,,,,,,,,,,,,,,,,,,,,
9599,240000.0,1,1,1,34,0,-2,-1,0,0,...,0.0,4557.0,7557.0,9275.0,0.0,0.0,4557.0,3000.0,4000.0,2282.0


## Functions

### Timing and tone

In [3]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    #Just print how many minutes and seconds have passed
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tone(a=1000, b=700, play_time_seconds=1, framerate=4410):
    #Make a sound! Useful while training models
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t)+np.sin(b*t), rate=framerate, autoplay=True)

### Balance

In [4]:
def balance(data):
    df = data.copy()
    for i in range(2,7):
        for j in range(2,i+1):
            if i!=j:
                for pb in ['PAY','BILL']:
                    df[f'{pb}_AMT{i}_{j}'] = df[f'{pb}_AMT{i}'] - df[f'{pb}_AMT{j}']
                    df[f'{pb}_AMT{i}_{j}'] = df[f'{pb}_AMT{i}'] / (df[f'{pb}_AMT{j}']+1e-10)
                    
            pay_cols = [x for x in df.columns if x.startswith('PAY_AMT')
                        and int(x[-1]) >= i and int(x[-1]) <= j]
            bill_cols = [x for x in df.columns if x.startswith('BILL_AMT') 
                         and int(x[-1]) >= i and int(x[-1]) <= j]
            
            df[f'BALANCE_ACUM_AMT{i}_{j}'] = (df[pay_cols].sum(axis=1) - 
                                              df[bill_cols].sum(axis=1))
            df[f'BALANCE_ACUM_PERC_AMT{i}_{j}'] = (df[pay_cols].sum(axis=1) / 
                                                   (df[bill_cols].sum(axis=1)+1e-10))
    return df

### Autoencoder

In [5]:
from keras.models import Model
from keras import metrics, Input
from keras.layers import Dense, LeakyReLU

def make_autoencoder(X_train, encoding_dim=32, batch_size=500, epochs=150):
    kmetrics = [metrics.RootMeanSquaredError(name='rms'),metrics.MeanAbsoluteError(name='mae')]   

    len_input_output = X_train.shape[-1]
    input_ = Input(shape=(len_input_output,))
    
    encoded = Dense(units=encoding_dim**2, activation=LeakyReLU())(input_)
    encoded = Dense(units=encoding_dim*2, activation=LeakyReLU())(encoded)
    bottleneck = Dense(units=encoding_dim, activation=LeakyReLU())(encoded)
    decoded = Dense(units=encoding_dim*2, activation=LeakyReLU())(bottleneck)
    decoded = Dense(units=encoding_dim**2, activation=LeakyReLU())(decoded)
    output = Dense(units=len_input_output, activation="linear")(decoded)
    
    autoencoder = Model(inputs=input_, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',metrics=[kmetrics])
    autoencoder.fit(X_train, X_train,batch_size=batch_size,epochs=epochs)
    
    encoder = Model(inputs=input_, outputs=bottleneck)
    return autoencoder, encoder

### R2

In [6]:
from tensorflow.keras import backend as K

def r2(y_true, y_pred):
    res =  K.sum(K.square(y_true-y_pred)) 
    tot = K.sum(K.square(y_true-K.mean(y_true))) 
    return (1-res/(tot+K.epsilon()))

### Pipeline

In [7]:
def full_pipeline(data,
                  encod_obj,prep_obj,
                  cat_cols,num_cols,
                  target_col,id_col='ACCOUNT_ID',
                  val=False):
    df = data.copy()
    encoder = encod_obj
    prep = prep_obj
    cat = cat_cols
    num = num_cols
    target = target_col
    
    df = balance(df)
    if val: y_test = None
    else: y_test = df[target].values
    X_test = df[cat+num].copy()
    X_test = pd.DataFrame(prep.transform(X_test),
                          index=X_test.index,columns=X_test.columns)
    X_encod = pd.DataFrame(encoder.predict(X_test), index=X_test.index)
    return X_test,X_encod,y_test

## Transform

### Train - test

In [8]:
from sklearn.model_selection import train_test_split

df, test = train_test_split(df,train_size=0.87,random_state=22)

### Variables

In [9]:
df = balance(df)
df.sample()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT6_4,BILL_AMT6_4,BALANCE_ACUM_AMT6_4,BALANCE_ACUM_PERC_AMT6_4,PAY_AMT6_5,BILL_AMT6_5,BALANCE_ACUM_AMT6_5,BALANCE_ACUM_PERC_AMT6_5,BALANCE_ACUM_AMT6_6,BALANCE_ACUM_PERC_AMT6_6
ACCOUNT_ID,,,,,,,,,,,,,,,,,,,,,
25086,180000.0,2,2,2,34,0,0,0,0,0,...,1.0,1.005484,0.0,0.0,1.086957,1.012982,0.0,0.0,-122419.0,0.039241


In [10]:
target = 'PAY_AMT1'
cat = ['SEX', 'EDUCATION', 'MARRIAGE','PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
num = [x for x in df.columns if x not in cat+[target]]

### Preprocessing

In [11]:
y_train = df[target].values
X_train = df[cat+num].copy()

In [12]:
'''!pip install category_encoders'''
from category_encoders.cat_boost import CatBoostEncoder
#Encoder para categóricas
encod = CatBoostEncoder()

from sklearn.preprocessing import MinMaxScaler
#Escala para numéricas
mm_x = MinMaxScaler()

from sklearn.compose import ColumnTransformer
#Se aplicará transformación para cada tipo de columnas
prep = ColumnTransformer(transformers=[('Cat_encod', encod, cat),
                                       ('Scale', mm_x, num)])

X_train = pd.DataFrame(prep.fit_transform(X_train,y_train),
                       index=X_train.index,columns=X_train.columns)

### Autoencoder

In [13]:
autoencoder, encoder = make_autoencoder(X_train)

Epoch 1/150
10/10 [==============================] - 16s 7ms/step - loss: 0.1953 - rms: 0.4390 - mae: 0.1656
Epoch 2/150
10/10 [==============================] - 0s 6ms/step - loss: 0.0512 - rms: 0.2263 - mae: 0.0980
Epoch 3/150
10/10 [==============================] - 0s 7ms/step - loss: 0.0370 - rms: 0.1924 - mae: 0.0771
Epoch 4/150
10/10 [==============================] - 0s 7ms/step - loss: 0.0280 - rms: 0.1672 - mae: 0.0667
Epoch 5/150
10/10 [==============================] - 0s 7ms/step - loss: 0.0192 - rms: 0.1384 - mae: 0.0556
Epoch 6/150
10/10 [==============================] - 0s 7ms/step - loss: 0.0153 - rms: 0.1235 - mae: 0.0507
Epoch 7/150
10/10 [==============================] - 0s 7ms/step - loss: 0.0106 - rms: 0.1031 - mae: 0.0440
Epoch 8/150
10/10 [==============================] - 0s 7ms/step - loss: 0.0075 - rms: 0.0868 - mae: 0.0402
Epoch 9/150
10/10 [==============================] - 0s 7ms/step - loss: 0.0054 - rms: 0.0732 - mae: 0.0353
Epoch 10/150
10/10 [=======

10/10 [==============================] - 0s 9ms/step - loss: 4.1517e-04 - rms: 0.0203 - mae: 0.0108
Epoch 75/150
10/10 [==============================] - 0s 9ms/step - loss: 3.7159e-04 - rms: 0.0193 - mae: 0.0102
Epoch 76/150
10/10 [==============================] - 0s 9ms/step - loss: 3.9515e-04 - rms: 0.0199 - mae: 0.0105
Epoch 77/150
10/10 [==============================] - 0s 10ms/step - loss: 3.5739e-04 - rms: 0.0189 - mae: 0.0101
Epoch 78/150
10/10 [==============================] - 0s 9ms/step - loss: 3.2293e-04 - rms: 0.0180 - mae: 0.0095
Epoch 79/150
10/10 [==============================] - 0s 9ms/step - loss: 3.3153e-04 - rms: 0.0182 - mae: 0.0096
Epoch 80/150
10/10 [==============================] - 0s 9ms/step - loss: 4.1797e-04 - rms: 0.0204 - mae: 0.0102
Epoch 81/150
10/10 [==============================] - 0s 9ms/step - loss: 4.7884e-04 - rms: 0.0218 - mae: 0.0117
Epoch 82/150
10/10 [==============================] - 0s 10ms/step - loss: 3.3095e-04 - rms: 0.0182 - mae: 0

10/10 [==============================] - 0s 11ms/step - loss: 1.9035e-04 - rms: 0.0138 - mae: 0.0074
Epoch 146/150
10/10 [==============================] - 0s 9ms/step - loss: 2.1584e-04 - rms: 0.0147 - mae: 0.0077
Epoch 147/150
10/10 [==============================] - 0s 9ms/step - loss: 2.8542e-04 - rms: 0.0169 - mae: 0.0095
Epoch 148/150
10/10 [==============================] - 0s 9ms/step - loss: 2.4042e-04 - rms: 0.0155 - mae: 0.0088
Epoch 149/150
10/10 [==============================] - 0s 9ms/step - loss: 1.9562e-04 - rms: 0.0140 - mae: 0.0077
Epoch 150/150
10/10 [==============================] - 0s 9ms/step - loss: 2.2845e-04 - rms: 0.0151 - mae: 0.0081


In [14]:
X_encod = pd.DataFrame(data=encoder.predict(X_train), index=X_train.index)
X_encod.sample(4)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
ACCOUNT_ID,,,,,,,,,,,,,,,,,,,,,
20433,0.781596,-0.142165,0.915651,1.150129,-0.150643,0.824371,0.249619,0.863940,0.456060,0.890578,...,0.638484,1.271279,1.139497,-0.136120,-0.044002,0.632589,-0.143171,-0.079651,0.824481,0.894210
3629,1.459184,-0.207628,0.248149,1.208892,-0.486989,1.519949,-0.009314,0.842084,0.399895,1.951291,...,2.028988,0.407866,1.134067,-0.225357,0.883929,0.773616,-0.443451,-0.094697,1.532923,1.814409
16761,0.661434,-0.154975,1.023674,1.052700,-0.139022,0.995192,0.171908,0.914518,0.276284,0.824062,...,0.619257,1.047031,1.112129,-0.086388,-0.062975,0.568713,-0.204223,-0.130844,0.924316,0.906232
9466,0.636628,-0.105243,0.768256,0.832389,-0.058672,0.565349,0.274920,0.925661,0.435359,0.907316,...,0.467779,1.131191,1.065002,-0.080786,-0.070051,0.420110,-0.160742,-0.099613,0.601304,0.745563


## Model

In [15]:
X_train_orig = X_train.copy()
X_train = X_encod.copy()

In [16]:
X_train = X_train_orig.copy()

### LinearReg

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

model_reg = LinearRegression()

print('Score: ',model_reg.fit(X_train,y_train).score(X_train,y_train))
#The most relevant features to the model_reg
coef = pd.DataFrame(zip(X_train.columns,model_reg.coef_)).sort_values(1,0,0).reset_index(drop=True)
coef.head().append(coef.tail())

Score:  0.38915443914816483


,0,1
0,BILL_AMT2,305764.475103
1,PAY_AMT2,202373.628553
2,BALANCE_ACUM_PERC_AMT3_3,86566.113537
3,BALANCE_ACUM_AMT3_3,81642.344937
4,PAY_AMT5,45539.110432
66,PAY_AMT3_2,-31173.151874
67,BALANCE_ACUM_PERC_AMT2_2,-63164.723437
68,BILL_AMT4_3,-88558.561200
69,BILL_AMT3,-94616.816517
70,BILL_AMT1,-151023.784132


### RandomForest

In [18]:
from sklearn.ensemble import RandomForestRegressor

model_forest = RandomForestRegressor(n_jobs=-1,random_state=22)

print(model_forest.fit(X_train,y_train).score(X_train,y_train))
#The most relevant features to the model_forest
pd.DataFrame(zip(X_train.columns,model_forest.feature_importances_)).sort_values(1,0,0).head(10).reset_index(drop=True)

0.9492804948987221


,0,1
0,BILL_AMT2,0.211284
1,BILL_AMT1,0.135210
2,PAY_AMT3,0.134490
3,PAY_2,0.092157
4,BALANCE_ACUM_AMT2_2,0.079659
5,PAY_AMT2,0.078999
6,PAY_AMT4,0.033316
7,BALANCE_ACUM_AMT3_3,0.030545
8,BILL_AMT4,0.027849
9,BALANCE_ACUM_AMT6_6,0.022240


### XGBoost

In [22]:
from xgboost.sklearn import XGBRegressor
xgb = XGBRegressor()

param_xgb = {'learning_rate':[x/100 for x in range(1,52,10)],
             'n_estimators':[x for x in range(100,1100,200)],
             'max_depth':[x for x in range(1,11,3)], 
             'min_child_weight':[x/10 for x in range(1,101,20)],
             'reg_lambda': [x/10 for x in range(1,101,20)]
            }
            
from sklearn.model_selection import RandomizedSearchCV
search_xgb = RandomizedSearchCV(param_distributions = param_xgb, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'r2', 
                                estimator = xgb, 
                                verbose = 1,
                                n_iter = 70,
                                random_state = 22)

model_xgb = Pipeline(steps=[('scaler', mm_x),
                            ('model', search_xgb)])

print(model_xgb.fit(X_train,y_train).score(X_train,y_train)
      
#The most relevant features to the model_xgb
pd.DataFrame(zip(X_train.columns,model_xgb[1].best_estimator_.feature_importances_)).sort_values(1,0,0).head(10).reset_index(drop=True)

,0,1
0,PAY_2,0.400338
1,BILL_AMT1,0.136858
2,BILL_AMT2,0.108436
3,PAY_AMT3,0.108028
4,PAY_AMT6,0.034243
5,PAY_AMT2,0.030058
6,BALANCE_ACUM_AMT2_2,0.022224
7,BALANCE_ACUM_AMT3_3,0.019237
8,BALANCE_ACUM_PERC_AMT6_6,0.016929
9,BALANCE_ACUM_PERC_AMT3_3,0.009740


### Test

In [23]:
model = model_xgb

X_test,X_test_encod,y_test = full_pipeline(test,encoder,prep,cat,num,target)
model.score(X_test,y_test)

0.6930860675325108

## Export

In [24]:
val = pd.read_csv(os.path.join(BASE_DIR,'val_PAY_AMT1.csv'),sep='|').set_index('ACCOUNT_ID')
X_val,X_val_encod,_ = full_pipeline(val,encoder,prep,cat,num,target,val=True)
final = X_val_encod.reset_index()[['ACCOUNT_ID']].join(pd.DataFrame(model.predict(X_val),
                                                                    columns=['y_hat']))
final.to_csv(os.path.join(BASE_DIR,'FloresHernandezEfrainIsmael_PAYAMT1.csv'),index=False)

In [25]:
import pickle

with open(os.path.join(BASE_DIR,'0504_payamt_model.pkl'), 'wb') as f:
    pickle.dump(model, f)

## End

In [26]:
time_exp(time.time() - start)
tone()

26 minutos con 52.25 segundos
